# Data Analyst au sein de la FAO

lien du brief : https://simplonline.co/briefs/688fc302-e4d4-4324-8904-f22316c31713



lien du dossier git : https://github.com/data-IA-2023/Groupe-3


Vous venez d'intégrer la FAO (Food and Agriculture Organization of the United Nations). BRAVO !!!!!!
Vous allez mener votre première mission en analysant les données mondiales de production agricole, d'alimentation et de nutrition.

**Livrables**

* Un dépôt Github
* Un notebook **fonctionnel, documenté et commenté**
* Une présentation écrite et orale résumant vos travaux, votre analyse et vos recommandations

**Contexte du projet**

* Au sein du service Data Analytics and Prospective de la FAO, vous êtes en charge d'analyser les données produites par les Nations Unies.

* Ces analyses conduisent à des rapports de synthèse à destination des décideurs.

* Pour êtes également en charge de la partie Prospective, c'est à dire donner des tendances et des évolutions probables quant à la production agricole, à l'alimentation et la nutrition mondiales.


**Ressource(s)**

FAO - Wikipedia : https://fr.wikipedia.org/wiki/Organisation_des_Nations_unies_pour_l'alimentation_et_l'agriculture

FAOSTAT : https://www.fao.org/faostat/fr/#home

Les données FAO 2013 : https://github.com/remijul/dataset/tree/master/fao_2013

## Partie traitement des données

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

#Chargement des données avec définition de l'index et changement des types des  dimensions
url_bd="https://github.com/remijul/dataset/raw/4156ed0c38c1f5e81b3bf8e6458a245c5fa6c35a/"
url_bd=""

chemein_BD_vegetal = url_bd+"fao_2013/FAOSTAT_2013_vegetal.csv"
FAOSTAT_2013_vegetal = pd.read_csv(chemein_BD_vegetal, sep=',',on_bad_lines='skip',dtype={'Code Pays': str,'Code Élément': str,'Code Produit':str, 'Année':str, 'Code Année':str})

chemein_BD = url_bd+"fao_2013\FAOSTAT_2013_animal.csv"
FAOSTAT_2013_animal = pd.read_csv(chemein_BD, sep=',',on_bad_lines='skip',dtype={'Code Pays': str,'Code Élément': str,'Code Produit':str, 'Code année':str, 'Année':str})

chemein_BD = url_bd+"fao_2013\FAOSTAT_2013_cereal.csv"
FAOSTAT_2013_cereal = pd.read_csv(chemein_BD, sep=',',on_bad_lines='skip',dtype={'Code Pays': str,'Code Élément': str,'Code Produit': str,'Code Année': str,'Année': str})

chemein_BD = url_bd+"fao_2013\FAOSTAT_2013_population.csv"
FAOSTAT_2013_population = pd.read_csv(chemein_BD, sep=',',on_bad_lines='skip',dtype={'Country Code': str,'Element Code': str,'Item Code': str,'Year Code': str,'Year': str})

chemein_BD = url_bd+"fao_2013\FAOSTAT_2013_sous_alimentation.csv"
FAOSTAT_2013_sous_alimentation = pd.read_csv(chemein_BD, sep=',',on_bad_lines='skip',dtype={'Code zone': str,'Code Élément': str,'Code Produit': str,'Code année': str,'Année': str})


### 1 - Etude guidée n°1

Il s’agit ici de répondre à toutes les questions ci-dessous.

#### Question 1 : Nombre d'humains sur la planète
Calculez le nombre total d’humains sur la planète.
Critiquez votre résultat. En cas d’anomalie, analysez et effectuer les corrections nécessaires.
Donnez le résultat de votre calcul pour l'année 2013.

In [7]:

Totalpopulation=FAOSTAT_2013_population['Value'].sum()*1000
print(f"Le nombre d'humains sur la planète est: {Totalpopulation}")

Le nombre d'humains sur la planète est: 8413993000


In [8]:
FAOSTAT_2013_population.drop(FAOSTAT_2013_population[FAOSTAT_2013_population['Flag']=='A'].index, inplace=True)
Totalpopulation=FAOSTAT_2013_population['Value'].sum()*1000
print(f"Le nombre d'humains sur la planète est: {Totalpopulation}")

Le nombre d'humains sur la planète est: 6997326000


In [ ]:
#Supprimer les agrégats

FAOSTAT_2013_animal.drop(FAOSTAT_2013_animal[FAOSTAT_2013_animal['Symbole']=='A'].index, inplace=True)
FAOSTAT_2013_cereal.drop(FAOSTAT_2013_cereal[FAOSTAT_2013_cereal['Symbole']=='A'].index, inplace=True)
FAOSTAT_2013_population.drop(FAOSTAT_2013_population[FAOSTAT_2013_population['Flag']=='A'].index, inplace=True)
FAOSTAT_2013_vegetal.drop(FAOSTAT_2013_vegetal[FAOSTAT_2013_vegetal['Symbole']=='A'].index, inplace=True)

#Pour sous population il n'y a pas de symbole alors en utilise Zone
FAOSTAT_2013_sous_alimentation.drop(FAOSTAT_2013_sous_alimentation[FAOSTAT_2013_sous_alimentation['Zone']=='Chine'].index, inplace=True)

#### Question 2 : Redondances
Identifiez ces redondances, en donnant votre réponse sous forme de formule mathématique (pas besoin de coder ici).
C'est une équation à 3 termes de type (a_1 + a2 + [...] = b_1 + b_2 + [...] = c_1 + c_2 + [...]) ) faisant intervenir chacune des 11 quantités données ci dessus.
Illustrez cette équation avec l'exemple du blé en France.

-Pour Les Data Animal, Cereal, Vegetal :

    Production + 'Importations - Quantité' + Variation de stock = 'Aliments pour animaux' + 'Autres Utilisations' + Nourriture + Pertes + Semences + Traitement + 'Exportations - Quantité'

    'Aliments pour animaux' + 'Autres Utilisations' + Nourriture + Pertes + Semences + Traitement = 'Disponibilité intérieure'

    'Disponibilité intérieure' + 'Exportations - Quantité' = Production + 'Importations - Quantité' + Variation de stock
    Production + Importations - Exportations + Variation de stock= Disponibilité intérieure

In [11]:
df_france=FAOSTAT_2013_cereal[FAOSTAT_2013_cereal['Pays']=='France']
df_france=df_france[df_france['Symbole']=='S']
df_france=df_france[df_france['Produit']=='Blé']


Production=df_france[df_france['Élément']=='Production']['Valeur'].sum()
Importations=df_france[df_france['Élément']=='Importations - Quantité']['Valeur'].sum()
Variation=df_france[df_france['Élément']=='Variation de stock']['Valeur'].sum()


Aliments_animaux=df_france[df_france['Élément']=='Aliments pour animaux']['Valeur'].sum()
Autres_Utilisations=df_france[df_france['Élément']=='Autres Utilisations']['Valeur'].sum()
Nourriture=df_france[df_france['Élément']=='Nourriture']['Valeur'].sum()
Pertes=df_france[df_france['Élément']=='Pertes']['Valeur'].sum()
Semences=df_france[df_france['Élément']=='Semences']['Valeur'].sum()
Traitement=df_france[df_france['Élément']=='Traitement']['Valeur'].sum()
Exportations=df_france[df_france['Élément']=='Exportations - Quantité']['Valeur'].sum()
Disponibilité=df_france[df_france['Élément']=='Disponibilité intérieure']['Valeur'].sum()

print(f"la somme de Production + Importations - Quantité' + Variation de stock est {Production+Importations+Variation}")
print(f"la somme de 'Aliments pour animaux' + 'Autres Utilisations' + Nourriture + Pertes + Semences + Traitement + 'Exportations - Quantité' est {Aliments_animaux+Autres_Utilisations+Nourriture+Pertes+Semences+Traitement+Exportations}")

print(f"Production + Importations - Exportations + Variation de stock= {Production+Importations-Exportations+Variation }")
print(f"Disponibilité = {Disponibilité}")

la somme de Production + Importations - Quantité' + Variation de stock est 41800.0
la somme de 'Aliments pour animaux' + 'Autres Utilisations' + Nourriture + Pertes + Semences + Traitement + 'Exportations - Quantité' est 41800.0
Production + Importations - Exportations + Variation de stock= 20298.0
Disponibilité = 20298.0


#### Question 3 : Disponibilité alimentaire (calories, protéines)
Calculez (pour chaque pays et chaque produit) la disponibilité alimentaire en kcal puis en kg de protéines. Vous ferez cela à partir de ces informations :
- Population de chaque pays
- Disponibilité alimentaire donnée pour chaque produit et pour chaque pays en kcal/personne/jour, -
- Disponibilité alimentaire en protéines donnée pour chaque produit et pour chaque pays en g/personne/jour.
Pour cette étape vous avez besoin de constituer une seule et même table à partir des tables animaux et végétaux.

#### Question 4 : Ratio énergie/poids
A partir de ces dernières informations, et à partir du poids de la disponibilité alimentaire (pour chaque pays et chaque produit), calculez pour chaque produit le ratio "énergie/poids", que vous donnerez en kcal/kg. Vous pouvez vérifier la cohérence de votre calcul en comparant ce ratio aux données disponibles sur internet, par exemple en cherchant la valeur calorique d'un oeuf.

#### Question 5 : Aliments les plus caloriques et protéiques
En considérant les aliments végétaux et animaux, citez 5 aliments parmi les 20 aliments les plus caloriques, en utilisant le ratio énergie/poids. Citez 5 aliments parmi les 20 aliments les plus riches en protéines.

#### Question 6 : Dispo. intérieure mondiale des végétaux
Calculez, pour les produits végétaux uniquement, la disponibilité intérieure mondiale exprimée en kcal.

#### Question 7 : Potentiel alimentaire des végétaux
Combien d'humains pourraient être nourris si toute la disponibilité intérieure mondiale de produits végétaux était utilisée pour de la nourriture ? Donnez les résultats en termes de calories, puis de protéines, et exprimez ensuite ces 2 résultats en pourcentage de la population mondiale.

#### Question 8 : Potentiel alimentaire des végétaux (destinés aux animaux pertes)
Combien d'humains pourraient être nourris si toute la disponibilité alimentaire en produits végétaux la nourriture végétale destinée aux animaux et les pertes de produits végétaux étaient utilisés pour de la nourriture ? Donnez les résultats en termes de calories, puis de protéines, et exprimez ensuite ces 2 résultats en pourcentage de la population mondiale.

#### Question 9 : Potentiel alimentaire de la dispo. Mondiale
Combien d'humains pourraient être nourris avec la disponibilité alimentaire mondiale ? Donnez les résultats en termes de calories, puis de protéines, et exprimez ensuite ces 2 résultats en pourcentage de la population mondiale.

#### Question 10 : Proportion de la sous-nutrition mondiale
A partir des données téléchargées qui concernent la sous-nutrition, répondez à cette question : Quelle proportion de la population mondiale est considérée comme étant en sous-nutrition ?

#### Question 11 : Céréales
Établissez la liste des produits (ainsi que leur code) considéré comme des céréales selon la FAO. En ne prenant en compte que les céréales destinées à l'alimentation (humaine et animale), quelle proportion (en termes de poids) est destinée à l'alimentation animale ?

#### Question 12 : Sous-nutrition
Sélectionnez parmi les données des bilans alimentaires les informations relatives aux pays dans lesquels la FAO recense des personnes en sous-nutrition.
Repérez les 15 produits les plus exportés par ce groupe de pays.
Parmi les données des bilans alimentaires au niveau mondial, sélectionnez les 200 plus grandes importations de ces produits (1 importation = une quantité d'un produit donné importée par un pays donné).
Groupez ces importations par produit, afin d'avoir une table contenant 1 ligne pour chacun des 15 produits. Ensuite, calculez pour chaque produit les 2 quantités suivantes :

    • le ratio entre la quantité destinés aux "Autres utilisations" (Other uses) et la disponibilité intérieure.
    • le ratio entre la quantité destinée à la nourriture animale et la quantité destinée à la nourriture (animale + humaine)
    
Donnez les 3 produits qui ont la plus grande valeur pour chacun des 2 ratios (vous aurez donc 6 produits à citer)

#### Question 13 : USA
Combien de tonnes de céréales pourraient être libérées si les USA diminuaient leur production de produits animaux de 10% ?

#### Question 14 : Thaïlande
En Thaïlande, quelle proportion de manioc est exportée ? Quelle est la proportion de personnes en sous-nutrition?

### 2 - Etude guidée n°2

Il s’agit ici d’intégrer toutes les données dans une base de données relationnelle puis de répondre à toutes les questions ci-dessous.
La base de données contiendra 4 tables : population, dispo_alim, equilibre_prod, sous_nutrition.

#### Question 15 : Table population
Définir une clé primaire pertinente pour la table population. Intégrer la table dans la base de données relationnelle.

#### Question 16 : Table dispo_alim
La table appelée dispo_alim contient pour chaque pays, pour chaque produit, sur l'année 2013, les informations suivantes:
- la nature du produit (deux valeurs possibles“animal” ou “végétal”)
- disponibilité alimentaire en tonnes
- disponibilité alimentaire en Kcal/personne/jour
- disponibilité alimentaire de protéines en g/personne/jour
- disponibilité alimentaire de matières grasses en g/personne/jour
La table contient ces colonnes : pays, code_pays, année, produit, code_produit, origin, dispo_alim_tonnes, dispo_alim_kcal_p_j, dispo_prot, dispo_mat_gr .
Définir une clé primaire pertinente pour la table dispo_alim. Intégrer la table dans la base de données relationnelle.

#### Question 17 : Table equilibre_prod
La table appelée equilibre_prod contient pour chaque pays, pour chaque produit, et pour l'année disponible 2013, les quantités suivantes :
- disponibilité intérieure
- aliments pour animaux
- semences
- pertes
- transformés
- nourriture
- autres utilisations
Elle contient ces colonnes : pays, code_pays, année, produit, code_produit, dispo_int, alim_ani, semences, pertes, transfo, nourriture, autres_utilisations.
Définir une clé primaire pertinente pour la table equilibre_prod. Intégrer la table dans la base de données relationnelle.

#### Question 18 : Table sous_nutrition
La table appelée sous_nutrition, contient le nombre de personnes en sous-alimentation pour chaque pays et pour chaque année. Elle contient 4 colonnes : pays, code_pays, année, nb_personnes.
Définir une clé primaire pertinente pour la table sous_nutrition. Intégrer la table dans la base de données relationnelle.

#### Question 19 : Requêtes SQL
Formuler les requêtes SQL pour répondre aux questions suivantes :
    • Quels sont les 10 pays ayant le plus haut ratio disponibilité alimentaire/habitant en termes de protéines (en kg) par habitant, puis en termes de kcal par habitant ?
    • Quels sont les 10 pays ayant le plus faible ratio disponibilité alimentaire/habitant en termes de protéines (en kg) par habitant ?
    • Quelle est la quantité totale (en kg) de produits perdus par pays?
    • Quels sont les 10 pays pour lesquels la proportion de personnes sous-alimentées est la plus forte ?
    • Quels sont les 10 produits pour lesquels le ratio Autres utilisations/Disponibilité intérieure est le plus élevé ?

#### Question 20 : Autres utilisations des produits
Pour quelques-uns des produits identifiés dans cette dernière requête SQL, supposez quelles sont ces "autres utilisations" possibles (recherchez sur internet !).

### 3 - Analyses complémentaire

Fort de votre expérience de Data Analyst, il s’agit maintenant de réaliser vos propres analyses et exploration.